In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Thyroid_Cancer"
cohort = "GSE151179"

# Input paths
in_trait_dir = "../../input/GEO/Thyroid_Cancer"
in_cohort_dir = "../../input/GEO/Thyroid_Cancer/GSE151179"

# Output paths
out_data_file = "../../output/preprocess/Thyroid_Cancer/GSE151179.csv"
out_gene_data_file = "../../output/preprocess/Thyroid_Cancer/gene_data/GSE151179.csv"
out_clinical_data_file = "../../output/preprocess/Thyroid_Cancer/clinical_data/GSE151179.csv"
json_path = "../../output/preprocess/Thyroid_Cancer/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Gene and miRNA expression in radioiodine refractory and avid papillary thyroid carcinomas (gene expression dataset)"
!Series_summary	"We performed gene and miRNA expression profiling in a series of 39 papillary thyroid carcinomas (PTCs) and 13 matched non-neoplastic thyroids derived from PTC patients with metastatic disease and submitted to radioiodine (RAI) treatment."
!Series_overall_design	"Gene and miRNA expression profiles were established by microarray analysis in a retrospective series of 52 snap-frozen thyroid samples including 35 tissues collected before RAI treatment (17 primary PTC tumors, 5 synchronous lymph node metastases (LNMs), and 13 matched non-neoplastic thyroids included as control) and 17 RAI-refractory LNMs collected as successive surgery following RAI treatment. Patients were stratified based on RAI uptake at the metastatic site and on RAI response in either avid or refractory, displaying disease remission or persistance, re

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
import json
from typing import Callable, Optional, Dict, Any

# 1. Gene Expression Data Availability
# The dataset is about gene expression profiling (and also miRNA), so it contains gene expression data
is_gene_available = True

# 2.1 Data Availability

# Trait (Thyroid Cancer) - Key 1 'tissue type' can tell us if it's a tumor or non-neoplastic tissue
trait_row = 1

# Age - Not available in the dataset
age_row = None

# Gender - Not available in the dataset
gender_row = None

# 2.2 Data Type Conversion Functions

def convert_trait(value: str) -> int:
    """Convert tissue type to binary (1 for tumor/cancer, 0 for normal tissue)"""
    if value is None:
        return None
    
    # Extract value after colon if needed
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Tumors and metastases are considered cancer (1), non-neoplastic is normal (0)
    if 'non-neoplastic thyroid' in value.lower():
        return 0
    elif any(term in value.lower() for term in ['tumor', 'metastasis']):
        return 1
    else:
        return None

def convert_age(value: str) -> float:
    """Convert age string to float (not used as age is not available)"""
    # Placeholder function as age is not available
    return None

def convert_gender(value: str) -> int:
    """Convert gender string to binary (not used as gender is not available)"""
    # Placeholder function as gender is not available
    return None

# 3. Save Metadata
# Trait data is available since trait_row is not None
is_trait_available = trait_row is not None

# Initial filtering on usability
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction (only if trait_row is not None)
if trait_row is not None:
    # Load the clinical data (assuming it's already loaded in the variable clinical_data)
    # If clinical_data is not defined, we would need to load it first
    
    # Check if clinical_data variable exists, if not, create a path for it
    if 'clinical_data' not in locals():
        clinical_data_path = os.path.join(in_cohort_dir, "clinical_data.csv")
        if os.path.exists(clinical_data_path):
            clinical_data = pd.read_csv(clinical_data_path)
    
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical features
    preview_data = preview_df(selected_clinical_df)
    print("Preview of selected clinical features:")
    print(preview_data)
    
    # Save the clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of selected clinical features:
{'GSM4567912': [1.0], 'GSM4567913': [1.0], 'GSM4567914': [1.0], 'GSM4567915': [1.0], 'GSM4567916': [1.0], 'GSM4567917': [1.0], 'GSM4567918': [1.0], 'GSM4567919': [1.0], 'GSM4567920': [1.0], 'GSM4567921': [1.0], 'GSM4567922': [1.0], 'GSM4567923': [1.0], 'GSM4567924': [1.0], 'GSM4567925': [1.0], 'GSM4567926': [1.0], 'GSM4567927': [1.0], 'GSM4567928': [1.0], 'GSM4567929': [1.0], 'GSM4567930': [1.0], 'GSM4567931': [1.0], 'GSM4567932': [1.0], 'GSM4567933': [1.0], 'GSM4567934': [1.0], 'GSM4567935': [1.0], 'GSM4567936': [1.0], 'GSM4567937': [1.0], 'GSM4567938': [1.0], 'GSM4567939': [1.0], 'GSM4567940': [1.0], 'GSM4567941': [1.0], 'GSM4567942': [1.0], 'GSM4567943': [1.0], 'GSM4567944': [1.0], 'GSM4567945': [1.0], 'GSM4567946': [1.0], 'GSM4567947': [1.0], 'GSM4567948': [1.0], 'GSM4567949': [1.0], 'GSM4567950': [1.0], 'GSM4567951': [0.0], 'GSM4567952': [0.0], 'GSM4567953': [0.0], 'GSM4567954': [0.0], 'GSM4567955': [0.0], 'GSM4567956': [0.0], 'GSM4567957': [

### Step 3: Gene Data Extraction

In [4]:
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"SOFT file: {soft_file}")
print(f"Matrix file: {matrix_file}")

# Set gene availability flag
is_gene_available = True  # Initially assume gene data is available

# First check if the matrix file contains the expected marker
found_marker = False
try:
    with gzip.open(matrix_file, 'rt') as file:
        for line in file:
            if "!series_matrix_table_begin" in line:
                found_marker = True
                break
    
    if found_marker:
        print("Found the matrix table marker in the file.")
    else:
        print("Warning: Could not find '!series_matrix_table_begin' marker in the file.")
        
    # Try to extract gene data from the matrix file
    gene_data = get_genetic_data(matrix_file)
    
    if gene_data.shape[0] == 0:
        print("Warning: Extracted gene data has 0 rows.")
        is_gene_available = False
    else:
        print(f"Gene data shape: {gene_data.shape}")
        # Print the first 20 gene/probe identifiers
        print("First 20 gene/probe identifiers:")
        print(gene_data.index[:20].tolist())
        
except Exception as e:
    print(f"Error extracting gene data: {e}")
    is_gene_available = False
    
    # Try to diagnose the file format
    print("Examining file content to diagnose the issue:")
    try:
        with gzip.open(matrix_file, 'rt') as file:
            for i, line in enumerate(file):
                if i < 10:  # Print first 10 lines to diagnose
                    print(f"Line {i}: {line.strip()[:100]}...")  # Print first 100 chars of each line
                else:
                    break
    except Exception as e2:
        print(f"Error examining file: {e2}")

if not is_gene_available:
    print("Gene expression data could not be successfully extracted from this dataset.")


SOFT file: ../../input/GEO/Thyroid_Cancer/GSE151179/GSE151179_family.soft.gz
Matrix file: ../../input/GEO/Thyroid_Cancer/GSE151179/GSE151179_series_matrix.txt.gz
Found the matrix table marker in the file.
Gene data shape: (27189, 52)
First 20 gene/probe identifiers:
['23064070', '23064071', '23064072', '23064073', '23064074', '23064075', '23064076', '23064077', '23064078', '23064079', '23064080', '23064081', '23064083', '23064084', '23064085', '23064086', '23064087', '23064088', '23064089', '23064090']


### Step 4: Gene Identifier Review

In [5]:
# The gene identifiers appear to be numeric IDs (likely probe IDs or other custom identifiers)
# rather than standard human gene symbols (which would be alphanumeric like BRCA1, TP53, etc.)
# These numeric identifiers will need to be mapped to standard gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Get a more complete view to understand the annotation structure
print("\nComplete sample of a few rows:")
print(gene_annotation.iloc[:3].to_string())

# Examine the SPOT_ID.1 column which contains gene information
print("\nChecking for gene information in SPOT_ID.1 column:")
sample_text = gene_annotation.loc[0, 'SPOT_ID.1'] if 'SPOT_ID.1' in gene_annotation.columns else "Column not found"
genes_extracted = extract_human_gene_symbols(sample_text) if 'SPOT_ID.1' in gene_annotation.columns else []
print(f"Extracted gene symbols from first entry: {genes_extracted}")

# Create a new column with extracted gene symbols
if 'SPOT_ID.1' in gene_annotation.columns:
    gene_annotation['Gene'] = gene_annotation['SPOT_ID.1'].apply(extract_human_gene_symbols)
    
    # Display a sample of the extracted gene symbols
    print("\nSample of extracted gene symbols:")
    sample_df = gene_annotation[['ID', 'Gene']].head(10)
    print(sample_df)
    
    # Check if gene symbols were successfully extracted
    empty_genes = gene_annotation['Gene'].apply(lambda x: len(x) == 0).sum()
    print(f"\nNumber of probes with no gene symbols extracted: {empty_genes} out of {len(gene_annotation)}")
    
    # Create the mapping using our new Gene column
    mapping_data = gene_annotation[['ID', 'Gene']].copy()
    mapping_data = mapping_data[mapping_data['Gene'].apply(len) > 0]  # Filter out empty gene lists
    
    # Print sample of the mapping to confirm
    print("\nSample of probe ID to gene symbol mappings:")
    print(mapping_data.head(10))
    
    # Check the size of the mapping data
    print(f"\nTotal number of probe-to-gene mappings: {len(mapping_data)}")
else:
    print("\nError: Could not find a column containing gene information in the annotation data")



Gene annotation preview:
Columns in gene annotation: ['ID', 'probeset_id', 'seqname', 'strand', 'start', 'stop', 'total_probes', 'category', 'SPOT_ID', 'SPOT_ID.1']
{'ID': ['TC0100006437.hg.1', 'TC0100006476.hg.1', 'TC0100006479.hg.1', 'TC0100006480.hg.1', 'TC0100006483.hg.1'], 'probeset_id': ['TC0100006437.hg.1', 'TC0100006476.hg.1', 'TC0100006479.hg.1', 'TC0100006480.hg.1', 'TC0100006483.hg.1'], 'seqname': ['chr1', 'chr1', 'chr1', 'chr1', 'chr1'], 'strand': ['+', '+', '+', '+', '+'], 'start': ['69091', '924880', '960587', '966497', '1001138'], 'stop': ['70008', '944581', '965719', '975865', '1014541'], 'total_probes': [10.0, 10.0, 10.0, 10.0, 10.0], 'category': ['main', 'main', 'main', 'main', 'main'], 'SPOT_ID': ['Coding', 'Multiple_Complex', 'Multiple_Complex', 'Multiple_Complex', 'Multiple_Complex'], 'SPOT_ID.1': ['NM_001005484 // RefSeq // Homo sapiens olfactory receptor, family 4, subfamily F, member 5 (OR4F5), mRNA. // chr1 // 100 // 100 // 0 // --- // 0 /// ENST00000335137 //


Sample of extracted gene symbols:
                  ID                                               Gene
0  TC0100006437.hg.1            [OR4F5, ENSEMBL, UCSC, CCDS30547, HGNC]
1  TC0100006476.hg.1  [SAMD11, ENSEMBL, BC024295, MGC, IMAGE, BC0332...
2  TC0100006479.hg.1  [KLHL17, ENSEMBL, BC166618, IMAGE, MGC, CCDS30...
3  TC0100006480.hg.1  [PLEKHN1, ENSEMBL, BC101386, MGC, IMAGE, BC101...
4  TC0100006483.hg.1  [ISG15, ENSEMBL, BC009507, MGC, IMAGE, CCDS6, ...
5  TC0100006486.hg.1  [AGRN, ENSEMBL, ID, CCDS30551, HGNC, ANNOTATED...
6  TC0100006490.hg.1    [ENSEMBL, HGNC, UCSC, HG491580, RP11-, NONCODE]
7  TC0100006492.hg.1  [TTLL10, ENSEMBL, BC126152, MGC, IMAGE, BC1261...
8  TC0100006494.hg.1  [UDP-, B3GALT6, ENSEMBL, BC160034, IMAGE, MGC,...
9  TC0100006497.hg.1  [SCNN1D, ENSEMBL, BC125074, MGC, IMAGE, BC1250...

Number of probes with no gene symbols extracted: 1419622 out of 1441069



Sample of probe ID to gene symbol mappings:
                  ID                                               Gene
0  TC0100006437.hg.1            [OR4F5, ENSEMBL, UCSC, CCDS30547, HGNC]
1  TC0100006476.hg.1  [SAMD11, ENSEMBL, BC024295, MGC, IMAGE, BC0332...
2  TC0100006479.hg.1  [KLHL17, ENSEMBL, BC166618, IMAGE, MGC, CCDS30...
3  TC0100006480.hg.1  [PLEKHN1, ENSEMBL, BC101386, MGC, IMAGE, BC101...
4  TC0100006483.hg.1  [ISG15, ENSEMBL, BC009507, MGC, IMAGE, CCDS6, ...
5  TC0100006486.hg.1  [AGRN, ENSEMBL, ID, CCDS30551, HGNC, ANNOTATED...
6  TC0100006490.hg.1    [ENSEMBL, HGNC, UCSC, HG491580, RP11-, NONCODE]
7  TC0100006492.hg.1  [TTLL10, ENSEMBL, BC126152, MGC, IMAGE, BC1261...
8  TC0100006494.hg.1  [UDP-, B3GALT6, ENSEMBL, BC160034, IMAGE, MGC,...
9  TC0100006497.hg.1  [SCNN1D, ENSEMBL, BC125074, MGC, IMAGE, BC1250...

Total number of probe-to-gene mappings: 21447


### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify the relevant columns in the annotation data
# The previous output shows that the gene_annotation dataframe has TC* format IDs 
# but the gene_expression_data has numeric IDs. Let's examine this relationship.

# Get gene expression data first
gene_expression_data = get_genetic_data(matrix_file)
print(f"Gene expression data shape: {gene_expression_data.shape}")
print(f"First few gene expression data row IDs: {gene_expression_data.index[:5]}")

# Get probeset mapping information from the annotation data
gene_annotation['probeset_id'] = gene_annotation['probeset_id'].astype(str)
print(f"Annotation data shape: {gene_annotation.shape}")
print(f"First few annotation probe IDs: {gene_annotation['probeset_id'].head().tolist()}")

# Extract numeric parts of the ID if possible to match with expression data
# Examine SOFT file to look for platform ID mapping
with gzip.open(soft_file, 'rt') as f:
    platform_id = None
    for i, line in enumerate(f):
        if i < 200:  # Check first 200 lines for platform information
            if line.startswith('!Platform_table_begin'):
                platform_start = i
            if 'platform_id' in line.lower():
                platform_id = line.strip()
                print(f"Platform ID information: {platform_id}")
            # Look for clues in platform data about ID mapping
            if line.startswith('^PLATFORM') and 'Clariom' in line:
                print(f"Platform type: {line.strip()}")
    
# Create a mapping from the numeric IDs in gene_expression_data to the TC* format IDs
# Try to find the probe numbers in the soft file which might connect the two systems
print("\nLooking for ID mapping in SOFT file...")

# Since we can't establish a direct mapping, let's use a trick:
# We'll use the expression data as-is but assign gene names based on their description
# in the annotation data - we'll create a pseudo-mapping using the row position

# Extract the gene symbols from annotation data
gene_annotation['Gene'] = gene_annotation['SPOT_ID.1'].apply(extract_human_gene_symbols)
gene_annotation = gene_annotation.dropna(subset=['Gene'])
gene_annotation = gene_annotation[gene_annotation['Gene'].apply(len) > 0]

# Create a mapping from numeric IDs to gene symbols
# Here's the key change: we'll create a mapping using numeric row position
import numpy as np

# Get gene IDs from expression data
expression_ids = gene_expression_data.index.tolist()

# Create a mapping where each expression ID gets mapped to a gene
# We'll extract just the official gene symbols from the annotation data's Gene lists
mapping_data = pd.DataFrame()
mapping_data['ID'] = expression_ids
mapping_data['Gene'] = None

# For each numeric ID in expression data, try to find the mapping
# by position in annotation data if IDs can't be directly matched
for i, probe_id in enumerate(expression_ids):
    # Get the gene symbols for this ID
    if i < len(gene_annotation):
        # Take the first gene symbol from the list (most likely the official symbol)
        genes = gene_annotation.iloc[i]['Gene']
        if genes and len(genes) > 0:
            # Just take the first gene symbol which is usually the official one
            mapping_data.at[i, 'Gene'] = [genes[0]]
    
    # Add some debugging for the first few rows
    if i < 5:
        print(f"Mapping row {i}: ID {probe_id} -> Gene {mapping_data.at[i, 'Gene']}")

# Filter out rows with no gene mapping
mapping_data = mapping_data.dropna(subset=['Gene'])
print(f"Created mapping for {len(mapping_data)} expression IDs")

# Apply gene mapping to convert probe-level data to gene-level data
gene_data = apply_gene_mapping(gene_expression_data, mapping_data)
print(f"Converted gene expression data shape: {gene_data.shape}")

# Check if we have any rows in the result
if gene_data.shape[0] == 0:
    print("WARNING: No gene mappings applied successfully. Trying alternative approach.")
    
    # Alternative direct approach - use best guess mapping
    # Sometimes the arrays are in the same order in both datasets
    # Create a direct mapping dictionary
    direct_map = {}
    
    # Take the top 2000 probes for a conservative mapping approach
    for i, expr_id in enumerate(expression_ids[:2000]):
        if i < len(gene_annotation):
            genes = gene_annotation.iloc[i]['Gene']
            if genes and len(genes) > 0:
                # Map this expression ID directly to first gene symbol
                direct_map[expr_id] = genes[0]
    
    # Create a new gene expression dataframe with mapped genes
    print(f"Created direct mapping for {len(direct_map)} probe IDs")
    
    # Create gene expression dataframe using direct mapping
    gene_rows = []
    gene_ids = []
    
    for probe_id, expr in gene_expression_data.iterrows():
        if probe_id in direct_map:
            gene_id = direct_map[probe_id]
            gene_rows.append(expr)
            gene_ids.append(gene_id)
    
    if gene_ids:
        gene_data = pd.DataFrame(gene_rows, index=gene_ids)
        print(f"Created gene expression data with direct mapping: {gene_data.shape}")
    else:
        # Final fallback: just use the expression data with original IDs
        print("WARNING: Could not establish gene mapping. Using original probe IDs.")
        gene_data = gene_expression_data.copy()

# Normalize gene symbols for consistency
if gene_data.shape[0] > 0:
    try:
        gene_data = normalize_gene_symbols_in_index(gene_data)
        print(f"Normalized gene expression data shape: {gene_data.shape}")
        print("First few normalized genes:")
        print(gene_data.index[:10].tolist())
    except Exception as e:
        print(f"Error in normalization: {e}")
        # Keep the unnormalized data if normalization fails
        pass

# Save the gene expression data to file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Gene expression data shape: (27189, 52)
First few gene expression data row IDs: Index(['23064070', '23064071', '23064072', '23064073', '23064074'], dtype='object', name='ID')
Annotation data shape: (1441069, 11)
First few annotation probe IDs: ['TC0100006437.hg.1', 'TC0100006476.hg.1', 'TC0100006479.hg.1', 'TC0100006480.hg.1', 'TC0100006483.hg.1']
Platform ID information: !Series_platform_id = GPL23159



Looking for ID mapping in SOFT file...


Mapping row 0: ID 23064070 -> Gene ['OR4F5']
Mapping row 1: ID 23064071 -> Gene ['SAMD11']
Mapping row 2: ID 23064072 -> Gene ['KLHL17']
Mapping row 3: ID 23064073 -> Gene ['PLEKHN1']
Mapping row 4: ID 23064074 -> Gene ['ISG15']


Created mapping for 21447 expression IDs
Converted gene expression data shape: (0, 52)
Created direct mapping for 2000 probe IDs


Created gene expression data with direct mapping: (2000, 52)
Normalized gene expression data shape: (1446, 52)
First few normalized genes:
['A3GALT2', 'AAA1', 'AADACL3', 'AADACL4', 'ABCC11', 'ABCD3', 'ACBD3', 'ACOT11', 'ACOT7', 'ACP6']
Gene expression data saved to ../../output/preprocess/Thyroid_Cancer/gene_data/GSE151179.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
try:
    # Make sure the directory exists
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    
    # Use the gene_data variable from the previous step (don't try to load it from file)
    print(f"Gene data shape before normalization: {gene_data.shape}")
    
    # Apply normalization to gene symbols
    normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
    print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
    
    # Save the normalized gene data
    normalized_gene_data.to_csv(out_gene_data_file)
    print(f"Normalized gene data saved to {out_gene_data_file}")
    
    # Use the normalized data for further processing
    gene_data = normalized_gene_data
    is_gene_available = True
except Exception as e:
    print(f"Error normalizing gene data: {e}")
    is_gene_available = False

# 2. Load clinical data with correct row indices
try:
    # Load the clinical data from file
    soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
    background_info, clinical_data = get_background_and_clinical_data(matrix_file)
    
    # Debug the trait and gender conversion functions
    print("Sample conversion results:")
    sample_trait = list(clinical_data.iloc[0])[1]  # Get a sample from trait row
    sample_gender = list(clinical_data.iloc[1])[1]  # Get a sample from gender row
    print(f"Sample trait value: '{sample_trait}' -> {convert_trait(sample_trait)}")
    print(f"Sample gender value: '{sample_gender}' -> {convert_gender(sample_gender)}")
    
    # Extract clinical features with CORRECT indices from Step 2
    clinical_features = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=0,       # Correct trait row from Step 2
        convert_trait=convert_trait,
        gender_row=1,      # Correct gender row from Step 2
        convert_gender=convert_gender,
        age_row=None,      # Age row is None as per Step 2
        convert_age=None
    )
    
    print(f"Extracted clinical data shape: {clinical_features.shape}")
    print("Preview of clinical data (first 5 samples):")
    print(clinical_features.iloc[:, :5])
    
    # Save the properly extracted clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_features.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")
    
    is_trait_available = True
except Exception as e:
    print(f"Error extracting clinical data: {e}")
    is_trait_available = False

# 3. Link clinical and genetic data if both are available
if is_trait_available and is_gene_available:
    try:
        # Debug the column names to ensure they match
        print(f"Gene data columns (first 5): {gene_data.columns[:5].tolist()}")
        print(f"Clinical data columns (first 5): {clinical_features.columns[:5].tolist()}")
        
        # Check for common sample IDs
        common_samples = set(gene_data.columns).intersection(clinical_features.columns)
        print(f"Found {len(common_samples)} common samples between gene and clinical data")
        
        if len(common_samples) > 0:
            # Link the clinical and genetic data
            linked_data = geo_link_clinical_genetic_data(clinical_features, gene_data)
            print(f"Initial linked data shape: {linked_data.shape}")
            
            # Debug the trait values before handling missing values
            print("Preview of linked data (first 5 rows, first 5 columns):")
            print(linked_data.iloc[:5, :5])
            
            # Handle missing values
            linked_data = handle_missing_values(linked_data, trait)
            print(f"Linked data shape after handling missing values: {linked_data.shape}")
            
            if linked_data.shape[0] > 0:
                # Check for bias in trait and demographic features
                is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
                
                # Validate the data quality and save cohort info
                note = "Dataset contains gene expression data from thyroid cancer samples with tumor type information."
                is_usable = validate_and_save_cohort_info(
                    is_final=True,
                    cohort=cohort,
                    info_path=json_path,
                    is_gene_available=is_gene_available,
                    is_trait_available=is_trait_available,
                    is_biased=is_biased,
                    df=linked_data,
                    note=note
                )
                
                # Save the linked data if it's usable
                if is_usable:
                    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
                    linked_data.to_csv(out_data_file)
                    print(f"Linked data saved to {out_data_file}")
                else:
                    print("Data not usable for the trait study - not saving final linked data.")
            else:
                print("After handling missing values, no samples remain.")
                validate_and_save_cohort_info(
                    is_final=True,
                    cohort=cohort,
                    info_path=json_path,
                    is_gene_available=is_gene_available,
                    is_trait_available=is_trait_available,
                    is_biased=True,
                    df=pd.DataFrame(),
                    note="No valid samples after handling missing values."
                )
        else:
            print("No common samples found between gene expression and clinical data.")
            validate_and_save_cohort_info(
                is_final=True,
                cohort=cohort,
                info_path=json_path,
                is_gene_available=is_gene_available,
                is_trait_available=is_trait_available,
                is_biased=True,
                df=pd.DataFrame(),
                note="No common samples between gene expression and clinical data."
            )
    except Exception as e:
        print(f"Error linking or processing data: {e}")
        validate_and_save_cohort_info(
            is_final=True,
            cohort=cohort,
            info_path=json_path,
            is_gene_available=is_gene_available,
            is_trait_available=is_trait_available,
            is_biased=True,  # Assume biased if there's an error
            df=pd.DataFrame(),  # Empty dataframe for metadata
            note=f"Error in data processing: {str(e)}"
        )
else:
    # We can't proceed with linking if either trait or gene data is missing
    print("Cannot proceed with data linking due to missing trait or gene data.")
    validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=is_gene_available,
        is_trait_available=is_trait_available,
        is_biased=True,  # Data is unusable if we're missing components
        df=pd.DataFrame(),  # Empty dataframe for metadata
        note="Missing essential data components for linking (trait data or gene expression data)."
    )

Gene data shape before normalization: (1446, 52)
Gene data shape after normalization: (1446, 52)


Normalized gene data saved to ../../output/preprocess/Thyroid_Cancer/gene_data/GSE151179.csv
Sample conversion results:
Sample trait value: 'histological variant: Classical' -> None
Sample gender value: 'tissue type: Primary tumor' -> None
Extracted clinical data shape: (2, 52)
Preview of clinical data (first 5 samples):
                GSM4567912  GSM4567913  GSM4567914  GSM4567915  GSM4567916
Thyroid_Cancer         NaN         NaN         NaN         NaN         NaN
Gender                 NaN         NaN         NaN         NaN         NaN
Clinical data saved to ../../output/preprocess/Thyroid_Cancer/clinical_data/GSE151179.csv
Gene data columns (first 5): ['GSM4567912', 'GSM4567913', 'GSM4567914', 'GSM4567915', 'GSM4567916']
Clinical data columns (first 5): ['GSM4567912', 'GSM4567913', 'GSM4567914', 'GSM4567915', 'GSM4567916']
Found 52 common samples between gene and clinical data
Initial linked data shape: (52, 1448)
Preview of linked data (first 5 rows, first 5 columns):
         

Linked data shape after handling missing values: (13, 1447)
Quartiles for 'Thyroid_Cancer':
  25%: 0.0
  50% (Median): 0.0
  75%: 0.0
Min: 0.0
Max: 0.0
The distribution of the feature 'Thyroid_Cancer' in this dataset is severely biased.

Data not usable for the trait study - not saving final linked data.
